In [6]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langchain_core.prompts import PromptTemplate

chat = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.1, max_tokens=300)

examples = [
    {
        "movie": "The Dark Knight",
        "answer": """Title: The Dark Knight
Director: Christopher Nolan
Main Cast: Christian Bale, Heath Ledger, Aaron Eckhart
Budget: $185 million
Box Office: $1.005 billion
Genre: Action, Crime, Drama
Synopsis: Batman faces the Joker, a chaotic criminal mastermind who forces Gotham to confront its deepest moral dilemmas.""",
    },
    {
        "movie": "Spirited Away",
        "answer": """Title: Spirited Away
Director: Hayao Miyazaki
Main Cast: Rumi Hiiragi, Miyu Irino
Budget: Unknown
Box Office: Unknown
Genre: Animation, Fantasy, Adventure
Synopsis: A young girl enters a mysterious spirit world and must find the courage to save her parents and return home.""",
    },
]

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "Can you explain about the {movie}"),
        ("ai","{answer}")
    ]
)

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples
)

final_prompt=ChatPromptTemplate([
    ("system", "You are a expert about movies."),
    example_prompt,
    ("human", "Can you explain about the {movie}")
])

chain = final_prompt | chat

chain.invoke({"movie": "Avatar"})

AIMessage(content="Title: Avatar\nDirector: James Cameron\nMain Cast: Sam Worthington, Zoe Saldana, Sigourney Weaver\nBudget: $237 million\nBox Office: $2.9 billion\nGenre: Science Fiction, Action, Adventure\nSynopsis: In the future, a paraplegic Marine is sent to the planet Pandora on a mission but becomes torn between following orders and protecting the indigenous Na'vi people.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 86, 'prompt_tokens': 194, 'total_tokens': 280, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-Cpybz5szoX6eY0iez7zwXPof7xrWn', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b4bd4-2e77-72d3-9c57-e3bc1338e2b9-0', usage